In [1]:
! pip install nltk
! pip install datasets
! pip install transformers[torch]
! pip install tokenizers
! pip install evaluate
! pip install rouge_score
! pip install sentencepiece
! pip install huggingface_hub

In [3]:
!pip install sentencepiece
!pip install transformers[sentencepiece]

   ---------------------------------------- 0.0/413.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/413.4 kB ? eta -:--:--
    --------------------------------------- 10.2/413.4 kB ? eta -:--:--
    --------------------------------------- 10.2/413.4 kB ? eta -:--:--
   -- ------------------------------------ 30.7/413.4 kB 262.6 kB/s eta 0:00:02
   -- ------------------------------------ 30.7/413.4 kB 262.6 kB/s eta 0:00:02
   ----- --------------------------------- 61.4/413.4 kB 297.7 kB/s eta 0:00:02
   ------ -------------------------------- 71.7/413.4 kB 302.7 kB/s eta 0:00:02
   ---------- --------------------------- 112.6/413.4 kB 437.6 kB/s eta 0:00:01
   ----------- -------------------------- 122.9/413.4 kB 423.5 kB/s eta 0:00:01
   ---------------- --------------------- 174.1/413.4 kB 523.5 kB/s eta 0:00:01
   -------------------- ----------------- 225.3/413.4 kB 599.0 kB/s eta 0:00:01
   -------------------------- ----------- 286.7/413.4 kB 681.0 kB/s eta 0:

In [22]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [23]:
from datasets import Dataset, DatasetDict

In [24]:
# Load the tokenizer, model, and data collator
MODEL_NAME = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# finetuned_model=model

In [ ]:
last_checkpoint = "/kaggle/input/checkpoint-4000/checkpoint-4000"

model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [25]:
import pandas as pd

csv_file_path = r'C:\Users\praya\Downloads\NLP\health_NLP_chatbot\datas\4.csv'
# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame
print(df.head())


                                               Query  \
0  After aremok;IVF and intercourse on the same d...   
1  What are the chances of conception after treat...   
2  Is there any chance of pregnancy after an egg ...   
3  How can one conceive successfully?.helo i am 2...   
4  How to confirm date of conception during pregn...   

                                              Answer  
0  u have good chances to get pregnant. ..u shoul...  
1  hello. it must be a tumour not sore like fibro...  
2  Hello. sometimes this does happen that when u ...  
3  Hi dear!I have read your question carefully an...  
4  hi there; I will suggest you the best possible...  


In [26]:
# converting it to dataset
dataset = Dataset.from_pandas(df)
dataset_dict = DatasetDict({'train': dataset})

In [27]:
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['Query', 'Answer'],
        num_rows: 16506
    })
})


In [28]:
# DATA_NAME = "yahoo_answers_qa"
# yahoo_answers_qa = load_dataset(DATA_NAME)

# print(yahoo_answers_qa)

In [29]:
yahoo_answers_qa = dataset_dict["train"].train_test_split(test_size=0.3)
# TODO: Change back variable name

In [30]:
yahoo_answers_qa

DatasetDict({
    train: Dataset({
        features: ['Query', 'Answer'],
        num_rows: 11554
    })
    test: Dataset({
        features: ['Query', 'Answer'],
        num_rows: 4952
    })
})

In [31]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question: "

# Define the preprocessing function

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [prefix + doc for doc in examples["Query"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)

   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=examples["Answer"],
                      max_length=512,
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

In [32]:
# Map the preprocessing function across our dataset
tokenized_dataset = yahoo_answers_qa.map(preprocess_function, batched=True)

Map: 100%|██████████| 4952/4952 [00:03<00:00, 1307.36 examples/s]


FLAN-T5 Training and Fine-Tuning
Before diving into the training process, it is better to identify the metrics to evaluate the overall performance of the fine-tuning.

Good evaluation metrics are important in any deep learning and machine learning project to evaluate the performance of models, not only during training but also later in production.

Two of the most common metrics to evaluate the performance of a text generation model are BLEU and ROUGE, and in this case, to evaluate the quality of an answer by comparing it to a reference answer.

The focus of this tutorial is ROUGE, but this wikipedia article provides more information about the BLEU score.



In [33]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

[nltk_data] Error loading punkt: <urlopen error [WinError 10054] An
[nltk_data]     existing connection was forcibly closed by the remote
[nltk_data]     host>


In [34]:
def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

   return result

## Training...

In [35]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 4
PER_DEVICE_EVAL_BATCH = 2
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)

In [36]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

In [1]:
trainer.train()

In [ ]:
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration
#last_checkpoint = "/kaggle/input/after-csv3/checkpoint-18000"
last_checkpoint = "/kaggle/input/after-csv1/checkpoint-9000"

finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)
 

In [ ]:
input = "How do I recover from depression? I am not able to do anything. I have been going through a lot of depression for 2 months. Would you advise me to go on a medication or sit idle at home?"

In [ ]:
inputs = tokenizer(input, return_tensors="pt")
# Increase max_length to generate a longer sequence
outputs = finetuned_model.generate(**inputs, max_length=120)  # Adjust max_length to desired length
answer = tokenizer.decode(outputs[0])
from textwrap import fill

print(fill(answer, width=80))